### Example: Phase separation
We setup a system of randomly dispersed `"A"` and `"B"` type particles, in equal proportions, run a short simulation, and examine the resulting trajectory.

In [10]:
import h5py
import numpy as np


def setup_system(box_size, number_density, out_path):
    """Setup a simple system of independent particles with no bonds
    
    Parameters
    ----------
    box_size : list of float
        Simulation box size in nm.
    number_density : float
        Number density per nm squared of particles in the simulation box.
    out_path : str
        Path of the created structure file.
    """
    box_size = np.asarray(box_size, dtype=np.float32)
    volume = np.prod(box_size)
    n_particles = int(number_density * volume)
    n_particles -= np.mod(n_particles, 2)
    
    with h5py.File(out_path, "w") as out_file:
        position_dataset = out_file.create_dataset(
            "coordinates",
            (1, n_particles, box_size.size,),
            dtype="float32",
        )
        types_dataset = out_file.create_dataset(
            "types",
            (n_particles,),
            dtype="i",
        )
        names_dataset = out_file.create_dataset(
            "names",
            (n_particles,),
            dtype="S10",
        )
        indices_dataset = out_file.create_dataset(
            "indices",
            (n_particles,),
            dtype="i",
        )
        
        types_dataset[:n_particles//2] = 0
        types_dataset[n_particles//2:] = 1
        names_dataset[:n_particles//2] = np.string_("A")
        names_dataset[n_particles//2:] = np.string_("B")
        indices_dataset[...] = np.arange(n_particles)
        position_dataset[...] = np.random.uniform(
            low=0.0, high=box_size, size=(1, n_particles, box_size.size),
        )

# 1042 particles in a 5nm x 5nm x 5nm simulation box.
setup_system([10, 10, 10], 8.333, "phase_separation_10x10x10nm.HDF5")

### Run the simulation
```bash
mpirun -n 6 python3 /Users/mortenledum/Documents/HyMD/hymd/__main__.py \
                    phase_separation.toml      \
                    phase_separation.HDF5      \
                    --seed 1                   \
                    --verbose 1
```

In [9]:
!ls -lha

total 1232
drwxr-xr-x@  9 mortenledum  staff   288B Dec 19 01:54 .
drwxr-xr-x@ 12 mortenledum  staff   384B Dec 19 01:47 ..
drwxr-xr-x@  3 mortenledum  staff    96B Dec 19 01:47 .ipynb_checkpoints
-rw-r--r--   1 mortenledum  staff    33K Dec 19 01:35 phase_separation.HDF5
-rw-r--r--   1 mortenledum  staff   3.7K Dec 19 01:54 phase_separation.ipynb
-rw-r--r--   1 mortenledum  staff   801B Dec 19 01:54 phase_separation.toml
-rw-r--r--   1 mortenledum  staff   187K Dec 19 01:54 sim.H5
-rw-r--r--   1 mortenledum  staff    70K Dec 19 01:54 sim.log
-rw-r--r--   1 mortenledum  staff   190K Dec 19 01:54 sim_chi=1.H5
